# Create `voc_sources.csv`

This notebook transforms the original VOC pay ledger (soldijboeken) file from the [VOC Opvarenden collection of the Dutch National Archives](https://www.nationaalarchief.nl/onderzoeken/index/nt00444) into a file with English column names and an additional column with normalized source type information.

## Environment Setup

Reload the Jupyter notebook extensions and import the pandas library for data manipulation.

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

## Load and Preview Data

Load the pay ledger data, add English column names using a helper function (`helper.py`), and preview the data.

In [ ]:
import helper

df = helper.read_voyages_df()

print(f'Pay ledger data\n\tnumber of rows: {df.shape[0]}\n\tnumber of columns: {df.shape[1]}')

df.head(5)

## Data Cleaning - Drop Specific Voyage

Drop the voyage number 1662.0 as it lacks necessary information and cannot be mapped to the Dutch-Asiatic Shipping database.

In [ ]:
df = df[df.das_voyage_num != '1662.0']

## Correct Voyage Numbers

Restore leading zeros dropped in voyage numbers and correct any typos in the voyage numbers using a helper function.

In [ ]:
df['das_voyage_num'] = df.das_voyage_num.apply(helper.add_voyage_number_leading_zeros)
# show that the leading zeros are restored
df[df.das_voyage_num == '0973.1']

from helper import voyage_num_map_str

df['das_voyage_num'] = df.das_voyage_num.apply(lambda x: voyage_num_map_str[x] if x in voyage_num_map_str else x)

# listing rows with das voyage number 0496.2 should give no results, 0496.1 should give one result
df[df.das_voyage_num.isin(['0496.2', '0496.1'])]

## Add Source Type and Clean Chamber Information

Introduce a new column `source_type` and refine the chamber field based on specific criteria for records associated with 'Regiment', 'Verzoekboeken', or other types.

This involves setting `source_type` to `regiment_book` for 'Regiment' records, to `request_book` for 'Verzoekboeken', and defaulting to `pay_ledger` for all others. The label Verzoekboeken is removed from the chamber name, and the chamber field is left blank for 'Regiment' records, indicating no chamber designation.

In [ ]:
df.chamber.value_counts()

#add source type
from helper import get_source_type

df['source_type'] = df.chamber.apply(get_source_type)
df.source_type.value_counts()

#clean chamber field
df['chamber'] = df.chamber.apply(lambda x: x.replace('Verzoekboeken',''))
df['chamber'] = df.chamber.apply(lambda x: None if 'Regiment' in x else x)
df.chamber.value_counts()

## Replace Voyage Number by DAS Voyage ID

Continue with steps to replace the voyage number with the DAS voyage ID and further data processing and cleaning.

This is needed as the voyage number is not in fact a number: some voyages have a suffix like 'a' or 'b'. A better alternative is the DAS Voyage ID from the [Dutch-Asiatic Shipping (DAS) database](https://resources.huygens.knaw.nl/das/). This is a proper running number, and is therefore easier to process in various applications. For voyages that do not appear in the DAS dataset, a placeholder voyage ID is added.

In [ ]:
#Retrieve the DAS Voyage ID from the DAS dataset (which can be found in the `external` folder).
das_file = '../external/das.xlsx'
das_df = pd.read_excel(das_file, engine='openpyxl', dtype={'voyId': int, 'voyNumberDAS': 'string'})
das_df.head(3)

#Rename the column `voyId` into `das_voyage_id`, and `voyNumberDAS` into `das_voyage_num`.
das_df = das_df[['voyId', 'voyNumberDAS']]
das_df.rename(columns={'voyId': 'das_voyage_id', 'voyNumberDAS': 'das_voyage_num'}, inplace=True)
das_df

#Merge the `das_voyage_id` into the original data frame.
merged_df = pd.merge(df, das_df, on='das_voyage_num', how='left')
merged_df

#Add a value in `das_voyage_id` for voyages that do not appear in the DAS dataset.
def fill_missing_voyage_ids(voyage_row):
    num_id_map = {
        '0000.0': 0,
        '4800.1': 100001,
        '4801.1': 100002,
    }
    if voyage_row['das_voyage_num'] in num_id_map:
        return num_id_map[voyage_row['das_voyage_num']]
    elif isinstance(voyage_row['das_voyage_num'], str):
        return voyage_row['das_voyage_id']
    else:
        return 100003

merged_df['das_voyage_id'] = merged_df.apply(fill_missing_voyage_ids, axis=1)

## Finalize Dataset

Remove unused columns and write the dataframe to a csv file.

In [ ]:
print(merged_df.columns)

In [ ]:
merged_df['das_voyage_id'] = merged_df.das_voyage_id.astype(int)

#Remove unused columns.
processed_headers = [
    'source_id',
    'ship_name',      
    'chamber',
    'source_type',
    'das_voyage_id',
    'remarks',
    'archival_reference',
    'uid'
]

drop_columns = [column for column in merged_df.columns if column not in processed_headers]
merged_df = merged_df.drop(drop_columns, axis=1)

processed_file = '../enriched/voc_sources.csv'

merged_df.to_csv(processed_file, index=None)